## Step 1: Import Libraries & Load Data

In this step, we import essential libraries such as pandas for data manipulation and load the CMS Open Payments dataset into a DataFrame. The dataset contains detailed information on payments made by manufacturers to healthcare providers. We start by reading the CSV file and displaying the first few rows to understand its structure.

In [ ]:
# Import necessary libraries
import pandas as pd

# Load the CMS Open Payments data (replace the path with your actual file)
df = pd.read_csv("/content/sample_data/data.csv", low_memory=False)

# Check first few rows
df.head()

,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_NPI,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,...,Associated_Drug_or_Biological_NDC_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Associated_Device_or_Medical_Supply_PDI_5,Program_Year,Payment_Publication_Date
0,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,720746,1.548429e+09,ISRAEL,NaN,SOKEYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,01/30/2025
1,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,720746,1.548429e+09,ISRAEL,NaN,SOKEYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,01/30/2025
2,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,720746,1.548429e+09,ISRAEL,NaN,SOKEYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,01/30/2025
3,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,125228,1.750340e+09,Jamie,NaN,Pelzel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,01/30/2025
4,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,605864,1.538104e+09,ROBERT,NaN,KESTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,01/30/2025


## Step 2: Filter Data
To ensure our analysis is focused and relevant, we filter the data using three key criteria:
1. payments made in the year 2023,
2. only payments classified as "Cash or cash equivalent",
3. payments made by companies headquartered in the United States.
These filters help us isolate direct cash transactions occurring in the U.S. healthcare system, which are most relevant for evaluating cash-pay clinic activity.

In [ ]:
# Convert payment date to datetime format
df["Date_of_Payment"] = pd.to_datetime(df["Date_of_Payment"], errors="coerce")

# Filter: 2023 records only
df_2023 = df[df["Date_of_Payment"].dt.year == 2023]

# Filter: Only "Cash or cash equivalent" payments
df_2023 = df_2023[
    df_2023["Form_of_Payment_or_Transfer_of_Value"] == "Cash or cash equivalent"
]

# Filter: Only payments made by companies based in the U.S.
df_2023 = df_2023[
    df_2023["Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country"]
    == "United States"
]

# Check remaining records
df_2023.shape

(13107, 91)

## Step 3: Select and Clean Necessary Columns

After applying the filters, we retain five relevant columns for analysis and traceability:

- `Date_of_Payment`: to confirm that all records are from 2023

- `Form_of_Payment_or_Transfer_of_Value`: ensures the payment type is cash

- `Covered_Recipient_License_State_code1`: used for grouping by state, indicating the state where the provider is licensed

- `Total_Amount_of_Payment_USDollars`: the key metric to measure monetary activity

- `Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country`: confirms the manufacturer is U.S.-based

We also drop any rows that have missing values in these columns to ensure data quality in subsequent visualizations and aggregations..

In [ ]:
# Keep all selected columns for later analysis
df_selected = df_2023[
    [
        "Date_of_Payment",  # For date verification
        "Form_of_Payment_or_Transfer_of_Value",  # Ensures all are cash-based
        "Covered_Recipient_License_State_code1",  # Core field for state-level grouping
        "Total_Amount_of_Payment_USDollars",  # Payment amount
        "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country",  # To show U.S. source
    ]
]

# Drop rows with missing state or payment value (required for analysis)
df_selected = df_selected.dropna(
    subset=["Covered_Recipient_License_State_code1", "Total_Amount_of_Payment_USDollars"]
)

# Preview the filtered dataset
df_selected.head()

,Date_of_Payment,Form_of_Payment_or_Transfer_of_Value,Covered_Recipient_License_State_code1,Total_Amount_of_Payment_USDollars,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country
0,2023-03-14,Cash or cash equivalent,MN,75.74,United States
1,2023-05-19,Cash or cash equivalent,MN,5.59,United States
2,2023-05-05,Cash or cash equivalent,MN,5.09,United States
5,2023-04-20,Cash or cash equivalent,MN,122.50,United States
6,2023-06-22,Cash or cash equivalent,MN,144.00,United States


## Step 4: Grouping and Aggregation by State
In this step, we group the filtered dataset by each provider's license state.
For each state, we calculate two key metrics:

- `Num_Records`: the total number of cash payment entries in 2023

- `Total_Payment`: the sum of all cash payments (in USD) to providers in that state

These values represent both the activity level and financial magnitude of cash-based healthcare transactions across states, laying the foundation for scoring and ranking.

In [ ]:
# Step 4: Group data by state and calculate number of records and total payment
state_agg = (
    df_selected.groupby("Covered_Recipient_License_State_code1")
    .agg(
        Num_Records=("Covered_Recipient_License_State_code1", "count"),
        Total_Payment=("Total_Amount_of_Payment_USDollars", "sum")
    )
    .reset_index()
    .rename(columns={"Covered_Recipient_License_State_code1": "State"})
)

# Preview the aggregated data (Top 5 rows)
state_agg.head()

,State,Num_Records,Total_Payment
0,AK,8,5754.81
1,AL,39,66749.89
2,AR,5,2609.45
3,AZ,300,6555135.84
4,CA,450,557042.90


## 🟦 Step 5: Normalization of State-Level Metrics

To make the two metrics (`Num_Records` and `Total_Payment`) comparable, we apply **Min-Max normalization**, a scaling technique that transforms raw values into a range between 0 and 1.  
This is especially useful when combining variables of different units or magnitudes into a single composite score.  

The Min-Max formula is:

<div align="center">
<strong>$ \text{Normalized Value} = \frac{X - X_{\min}}{X_{\max} - X_{\min}} $</strong>
</div>

where $X$ is the original value, $X_{\min}$ and $X_{\max}$ are the minimum and maximum values in the column.  

The resulting columns — `Norm_Records` and `Norm_Payment` — now represent each state's relative position in terms of **record volume** and **total cash payment value**.  
These scaled metrics serve as equal-weighted inputs for computing a combined **Hotness Score** in the following step.


In [ ]:
# Step 5: Normalize Num_Records and Total_Payment using Min-Max scaling
state_agg["Norm_Records"] = (
    (state_agg["Num_Records"] - state_agg["Num_Records"].min()) /
    (state_agg["Num_Records"].max() - state_agg["Num_Records"].min())
)

state_agg["Norm_Payment"] = (
    (state_agg["Total_Payment"] - state_agg["Total_Payment"].min()) /
    (state_agg["Total_Payment"].max() - state_agg["Total_Payment"].min())
)

# Preview the normalized dataset with 5 columns
state_agg[["State", "Num_Records", "Total_Payment", "Norm_Records", "Norm_Payment"]].head()

,State,Num_Records,Total_Payment,Norm_Records,Norm_Payment
0,AK,8,5754.81,0.000671,0.000147
1,AL,39,66749.89,0.003642,0.001705
2,AR,5,2609.45,0.000383,0.000066
3,AZ,300,6555135.84,0.028654,0.167475
4,CA,450,557042.90,0.043028,0.014232


In [ ]:
# Save the Step 5 result to the specified path
output_path = "/content/sample_data/state_agg.csv"
state_agg.to_csv(output_path, index=False)

# Confirm the file path
print(f"Step 5 CSV saved at: {output_path}")

Step 5 CSV saved at: /content/sample_data/state_agg.csv
